# PREVISÃO DA VELOCIDADE DO VENTO A CURTO PRAZO USANDO REDES NEURAIS QUÂNTICAS EM MUCURI PARA 3 HORAS SIMULTÂNEAS, BAHIA


## Import

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pennylane as qml
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import backend, optimizers, activations
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

from math import sqrt
from scipy import stats
from datetime import datetime,timedelta

## Carregando os Dados

In [42]:
def carregar_tabela(arquivo):
    # Carregando dataset
    dataset_train = pd.read_csv(arquivo, sep='\t', header = 0)

    # Separando os valores entre dados de entrada e dados a serem preditos 
    # (X e Y) Utilizado apenas a coluna de velocidade e removido o primeiro
    # índice para prever a próxima velocidade
    y_train_all = dataset_train[:].drop(dataset_train.index[0])
    # Remove a ultima linha do X pois não o predito Y não terá uma linha a mais
    X_train_all = dataset_train.iloc[:-3,:]
    

    y_train_all['1h - Vento'] = y_train_all.iloc[:,4].shift(0)
    y_train_all['2h - Vento'] = y_train_all.iloc[:,4].shift(-1)
    y_train_all['3h - Vento'] = y_train_all.iloc[:,4].shift(-2)
    y_train_all = y_train_all.iloc[:-2,-3:]
    
    return X_train_all,y_train_all.values

In [43]:
filename = 'train150_mucuri.txt'

X_train_all,y_train_all = carregar_tabela(filename)


print("Shape X", X_train_all.shape)
print("Shape y\n", y_train_all.shape)

print("y dataset:\n", y_train_all[:5])

X_train_all.head()

Shape X (547, 9)
Shape y
 (547, 3)
y dataset:
 [[12.72608696 12.08111113 11.64722224]
 [12.08111113 11.64722224 11.06444444]
 [11.64722224 11.06444444 10.32444445]
 [11.06444444 10.32444445  9.86277779]
 [10.32444445  9.86277779  9.59888887]]


,Dia,Mês,Ano,Hora,Velocidade,Direção,Temperatura,Umidade,Pressão
0,30,11,2015,14,13.012139,75.105481,27.516129,72.930636,1020.422601
1,30,11,2015,15,12.726087,68.334332,27.238095,75.212121,1020.394348
2,30,11,2015,16,12.081111,64.457865,27.105263,75.741379,1020.508333
3,30,11,2015,17,11.647222,53.842100,26.305556,75.302632,1020.611000
4,30,11,2015,18,11.064444,53.945279,25.464286,76.592593,1020.866500


# Normalizando

In [44]:
scaler_x = MinMaxScaler(feature_range=(-1, 1))
X_train_scaled = scaler_x.fit_transform(X_train_all)

scaler_y = MinMaxScaler(feature_range=(-1, 1))
y_train_scaled = scaler_y.fit_transform(y_train_all)

In [45]:
X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train_scaled, test_size=0.2)

# Rede Quântica 

In [ ]:
def H_layer(n_qubits):
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)

def Data_AngleEmbedding_layer(inputs, n_qubits):
    qml.templates.AngleEmbedding(inputs,rotation='Y', wires=range(n_qubits))

def RY_layer(w):
    print(w.shape)
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def ROT_layer(w):
    for i in range(5):
        qml.Rot(*w[i],wires=i)

def strong_entangling_layer(nqubits):
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[3,4])
    qml.CNOT(wires=[4,0])
    
    
def entangling_layer(nqubits):
    for i in range(0, nqubits - 1, 2): 
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  
        qml.CNOT(wires=[i, i + 1])

In [ ]:
n_qubits = 5
n_layers = 1

#dev = qml.device('lightning.qubit', wires=n_qubits)
dev = qml.device('default.qubit', wires=n_qubits)
@qml.qnode(dev)
def qnode(inputs, weights_1):
    H_layer(n_qubits)
    Data_AngleEmbedding_layer(inputs, n_qubits)
    for k in range(n_layers):
        entangling_layer(n_qubits)
        ROT_layer(weights_1[k])
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
weight_shapes = {"weights_1": (n_layers,5,3)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)
Activation=tf.keras.layers.Activation(activations.linear)
clayer_2 = tf.keras.layers.Dense(1,kernel_initializer='normal')
model = tf.keras.models.Sequential([qlayer,Activation, clayer_2])
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(opt, loss="mse")

## Modelo

<img src="neural network.png">

In [ ]:
tf.keras.backend.clear_session()

def create_MLP_model(neurons, activation_function, forecast_range=3, optimizer=tf.keras.optimizers.SGD(learning_rate = 0.01)):
    entrada = tf.keras.layers.Input(shape=(9,))
    networks = []
    for i in range(forecast_range):
        l = tf.keras.layers.Dense(neurons[0], activation=activation_function)(entrada)
        for j in range(len(neurons)):
            l = tf.keras.layers.Dense(neurons[j], activation=activation_function)(l)
        l = tf.keras.layers.Dense(1, activation=activation_function)(l)
        print(l)
        networks.append(l)
    total = tf.keras.layers.concatenate(networks)
    saida = tf.keras.layers.Dense(forecast_range, activation=activation_function)(total)
    model = tf.keras.Model(inputs=entrada, outputs=saida)
    model.compile(loss=['mse'], optimizer=optimizer, metrics=['mae'])
    return model

In [ ]:
model = create_MLP_model(neurons=[16,8], activation_function='tanh');
model.summary()

### Criando o Compilador e Executando o treino

<table>
<tr>
    <th><p align="left">Variável</p></th>
    <th><p align="left">Valor</p></th>
<tr>
    <td><p align="left">Loss</p></td>
    <td><p align="left">MSE (Mean Square Error)</p></td>
</tr>
<tr>
    <td><p align="left">Optimizer</p></td>
    <td><p align="left">SGD</p></td>
</tr>
<tr>
    <td><p align="left">Metrics</p></td>
    <td><p align="left">MAE (Mean Absolute Error)</p></td>
</tr>
<tr>
    <td><p align="left">Epochs</p></td>
    <td><p align="left">1000</p></td>
</tr>
<tr>
    <td><p align="left">Batch_Size</p></td>
    <td><p align="left">16</p></td>
</tr>
<tr>
    <td><p align="left">Verbose</p></td>
    <td><p align="left">2 (Exibir apenas o Epoch com o Loss e a Metric)</p></td>
</tr>
<tr>
    <td><p align="left">tx (Taxa de Aprendizado)</p></td>
    <td><p align="left">0.01</p></td>
</tr>
</table>

In [ ]:
# Fit retorna histórico do modelo
history_model = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_data=(X_val, y_val))

In [23]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=6, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
re=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='min', min_lr=0.00001)
fitting = model.fit(X_train, y_train, epochs=30, batch_size=1, validation_split=0.1, callbacks=[re], verbose=1)

Epoch 1/30
393/393 [==============================] - 1s 2ms/step - loss: 0.1371 - mae: 0.2878 - val_loss: 0.0567 - val_mae: 0.1914 - lr: 0.0100
Epoch 2/30
393/393 [==============================] - 1s 1ms/step - loss: 0.0870 - mae: 0.2343 - val_loss: 0.0507 - val_mae: 0.1817 - lr: 0.0100
Epoch 3/30
393/393 [==============================] - 1s 1ms/step - loss: 0.0842 - mae: 0.2278 - val_loss: 0.0435 - val_mae: 0.1659 - lr: 0.0100
Epoch 4/30
393/393 [==============================] - 1s 1ms/step - loss: 0.0812 - mae: 0.2251 - val_loss: 0.0430 - val_mae: 0.1665 - lr: 0.0100
Epoch 5/30
393/393 [==============================] - 1s 1ms/step - loss: 0.0805 - mae: 0.2225 - val_loss: 0.0509 - val_mae: 0.1828 - lr: 0.0100
Epoch 6/30
393/393 [==============================] - 1s 1ms/step - loss: 0.0795 - mae: 0.2208 - val_loss: 0.0443 - val_mae: 0.1709 - lr: 0.0100
Epoch 7/30
364/393 [==========================>...] - ETA: 0s - loss: 0.0787 - mae: 0.2177
Epoch 00007: ReduceLROnPlateau reducing

### Criando gráfico de Loss por Epoch

In [ ]:
def plot_history(history):
    plt.figure(figsize=(14,5), dpi=320, facecolor='w', edgecolor='k')
    plt.title("Loss")
    plt.xlabel("Época")
    plt.ylabel("Loss")
    plt.plot(history.history['loss'], label="Loss/Epoch")
    plt.plot(history.history['val_loss'], label="Val Loss/Epoch")
    plt.legend()
    plt.show()

In [ ]:
plot_history(history_model)

## Teste
### Carregando o Teste e executando a comparação entre original e predito

In [25]:
test = 'prev150_mucuri.txt'
X_test_all,y_test_all = carregar_tabela(test)
X_test_scaled = scaler.transform(X_test_all)
y_test_scaled = scaler_y.transform(y_test_all)

In [ ]:
predito = model.predict(X_test_scaled)

In [ ]:
predito_normal = scaler_y.inverse_transform(predito)


In [36]:
y_predict=model.predict(X_test_scaled,verbose=1)
#y_predict=y_predict.T
#y_predict=y_predict[0]

6/6 [==============================] - 0s 1ms/step


In [38]:
from sklearn import metrics
print('Mean Squared Error:', metrics.mean_squared_error(y_test_scaled, y_predict))

Mean Squared Error: 0.07086386471545147


### Calculando o intervalo de confiança do erro de predição do modelo

In [ ]:
def get_error_interval(model, X_val, Y_val, X_test, Y_test, y_test_pred, p_value):
    y_val_pred = model.predict(X_val)
    y_val_error = np.abs(Y_val - y_val_pred)
    error_quantile=np.ndarray((1,Y_val.shape[1]));
    for i in range(Y_val.shape[1]):
        error_quantile[0,i] = np.quantile(y_val_error[:,i], q=p_value, interpolation='higher')
        
    y_test_interval_pred_left=np.ndarray(y_test_pred.shape);
    y_test_interval_pred_right=np.ndarray(y_test_pred.shape);
    
    for i in range(y_test_pred.shape[1]):
        y_test_interval_pred_left[:,i] = y_test_pred[:,i] - error_quantile[0,i]
        y_test_interval_pred_right[:,i] = y_test_pred[:,i] + error_quantile[0,i]
    return error_quantile, y_test_interval_pred_left, y_test_interval_pred_right

In [ ]:
def get_mean_left_right_error_interval(model, y_scaler, y_test_pred):
    error, error_left, error_right = get_error_interval(model, X_val, y_val, X_test_scaled, y_test_scaled, y_test_pred, 0.95)
    
    error_left_normal = y_scaler.inverse_transform(error_left)
    error_right_normal = y_scaler.inverse_transform(error_right)

    mean_error_normal=np.ndarray((1,y_test_all.shape[1]));
    mean_error_left_normal=np.ndarray((1,y_test_all.shape[1]));
    mean_error_right_normal=np.ndarray((1,y_test_all.shape[1]));
    mean_predictions=np.ndarray((1,y_test_pred.shape[1]));

    for i in range(y_test_all.shape[1]):
        mean_error_left_normal[0,i] = np.mean(error_left_normal[:,i])
        mean_error_right_normal[0,i] = np.mean(error_right_normal[:,i])
        mean_predictions[0,i]=np.mean(y_test_pred[:,i])

    mean_error_normal=(mean_error_right_normal-mean_error_left_normal)/2
    return mean_predictions, mean_error_normal, mean_error_left_normal, mean_error_right_normal

In [ ]:
mean_predictions, mean_error_normal, mean_error_left_normal, mean_error_right_normal = get_mean_left_right_error_interval(
    model, scaler_y, predito_normal)


### Plotando os gráficos de predição versus observado no teste, junto com as métricas

### Função de Estatística

In [ ]:
def factor_of_2(y_true, y_pred):
    min_ = 0.5
    max_ = 2.0

    tensor_true = tf.constant(y_true)
    tensor_true = tf.cast(tensor_true, tf.float32)
    tensor_pred = tf.constant(y_pred)
    tensor_pred = tf.cast(tensor_pred, tf.float32)

    division = tf.divide(tensor_pred, tensor_true)

    greater_min = tf.greater_equal(division, min_)
    less_max = tf.less_equal(division, max_)

    res = tf.equal(greater_min, less_max)
    res = tf.cast(res, tf.float32)

    return backend.get_value(tf.reduce_mean(res))


def allmetrics(original,predito):
    r_value = 0
    slope, intercept, r_value, p_value, std_err = stats.linregress(original, predito)
    mse = mean_squared_error(original, predito)
    mae = mean_absolute_error(original, predito)
    rr = r2_score(original,predito)
    pea = stats.pearsonr(original, predito)
    fat = factor_of_2(original,predito)
    nmse = mse/stats.tvar(original)
    rmse = sqrt(mse)
    nrmse = rmse/stats.tstd(original)
    return mae,mse,nmse,r_value,rr,fat,rmse,nrmse

In [ ]:
def get_plot_prediction_versus_observed(model):
    valores = []
    for i in range(y_test_all.shape[1]):
        mae,mse,nmse,r_value,rr,fat,rmse,nrmse = allmetrics(y_test_all[:,i],predito_normal[:,i])
        valores.append([str(i+1)+" hora",mae,mse,nmse,rmse,nrmse,r_value,rr,fat,mean_error_normal[0,i],mean_error_left_normal[0,i],mean_predictions[0,i],mean_error_right_normal[0,i]])
        print("MAE:",mae)
        print("MSE:",mse)
        print("NMSE:",nmse)
        print("RMSE:",rmse)
        print("NRMSE:",nrmse)
        print("R:",r_value)
        print("R²:",rr)
        print("Fator de 2:",fat)

        plt.figure(figsize=(20,5), dpi=320, facecolor='w', edgecolor='k')
        plt.title("Previsão do vento para "+str(i+1)+" hora(s) à frente")
        plt.xlabel("Amostras")
        plt.ylabel("Velocidade do Vento (m/s)")
        plt.plot(predito_normal[:,i], label="Predito", color='blue')
        plt.fill_between(range(predito_normal.shape[0]), predito_normal[:,i]-mean_error_normal[0,i], predito_normal[:,i]+mean_error_normal[0,i], color='blue', alpha=0.05)
        plt.plot(y_test_all[:,i], label="Original", color='orange')
        plt.legend()
        plt.show()
    erros = pd.DataFrame(valores)
    erros.columns = ['Horas à frente','MAE','MSE','NMSE','RMSE','NRMSE','R','R²','Fator de 2', 'error interval (+/-)', 'left limit', 'mean', 'right limit']
    erros = erros.set_index('Horas à frente')
    erros.loc['Média'] = erros.mean()
    return erros;


In [ ]:
erros_pd=get_plot_prediction_versus_observed(model)
erros_pd

### Função para verificar se as predições de dois modelos diferentes sobre o mesmo dataset possui diferença estatística, i.e. são equivalentes ou diferentes

In [ ]:
from scipy.stats import wilcoxon
# sources: 
#   https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html
#   https://pythonfordatascienceorg.wordpress.com/wilcoxon-sign-ranked-test-python/
#   https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
# em resumo: o teste de Wilcoxon signed-rank testa a hipótese nula de que duas amostras pareadas relacionadas vêm da mesma distribuição. Ela é não paramétrica.
def verify_distribution_wilcoxtest(data1, data2, p_H0):
    stat, p = wilcoxon(data1, data2)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    if p > p_H0:
        print('Same distribution (fail to reject H0)')
    else:
        print('Different distribution (reject H0)')
    return stat, p

### Verificando se há diferença estatística entre o observado e o predito

In [ ]:
verify_distribution_wilcoxtest(y_test_all[:,0],predito_normal[:,0], 0.05)

### Criando novo modelo e verificando se suas predições têm ou não diferença estatística entre o modelo anterior

In [ ]:
# cria e ajusta outra rede MLP para comparar com a anterior
model2 = create_MLP_model(neurons=[32,16], activation_function='relu', optimizer=tf.keras.optimizers.RMSprop());
model2.summary()


In [ ]:
history_model2 = model2.fit(X_train, y_train, epochs=50, batch_size=64, verbose=1, validation_data=(X_val, y_val))

In [ ]:
predito2 = model2.predict(X_test_scaled)
predito2_normal = scaler_y.inverse_transform(predito2)
plot_history(history_model2)

In [ ]:
verify_distribution_wilcoxtest(predito_normal[:,0],predito2_normal[:,0], 0.05)

In [ ]:
verify_distribution_wilcoxtest(predito_normal[:,1],predito2_normal[:,1], 0.05)

In [ ]:
verify_distribution_wilcoxtest(predito_normal[:,2],predito2_normal[:,2], 0.05)

In [ ]:
erros2_pd=get_plot_prediction_versus_observed(model2)
erros2_pd